In [2]:
import sqlite3
import datetime
from faker import Faker
import random
import pandas as pd

In [3]:
faker = Faker('ru_ru')

In [4]:
class DatabaseInterface:
    def __init__(self, db_name: str):
        self.database = sqlite3.connect(db_name)
        self.cursor = self.database.cursor()
        
        self.valid_tables = ['Users', 'Orders']
        
        self.cursor.execute("PRAGMA foreign_keys=ON")
        self.database.commit()
        
        self.cursor.execute("""CREATE TABLE IF NOT EXISTS Users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_name VARCHAR(60) NOT NULL,
            age INTEGER,
            email VARCHAR
        )
        """)
        self.database.commit()
        
        self.cursor.execute("""CREATE TABLE IF NOT EXISTS Orders (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER REFERENCES Users(id) ON DELETE CASCADE,
            order_date VARCHAR(19),
            amount INTEGER
        )
        """)
        self.database.commit()
        
    def __del__(self):
        self.database.close()
        
    def query(self, query):
        try:
            response = self.cursor.execute(query)
            return response
        except Exception as e:
            print(f"Error has occured in query method: {e}")
            return e
        
    def __insert_into_users(self, name: str, age: int, email: str):
        try:
            assert isinstance(age, int)
            
            self.cursor.execute(f"INSERT INTO Users (user_name, age, email) VALUES('{name}', {age}, '{email}')")
            self.database.commit()
        except Exception as e:
            print("Error in insert_into_users method:", e)
    
    def __insert_into_orders(self, user_id: int, date: datetime.date, amount: int):
        try:
            self.cursor.execute(f"INSERT INTO Orders (user_id, order_date, amount) VALUES({user_id}, '{str(date)}', {amount})")
            self.database.commit()
        except Exception as e:
            print("Error in insert_into_orders method:", e)
            
    def insert_factory(self, table_name: str, *args):
        if table_name not in self.valid_tables:
            print(f"Error in  insert_factory: {table_name} table is not valid")
            return
        
        match table_name:
            case "Users":
                try:
                    self.__insert_into_users(*args)
                except Exception as e:
                    print("Error in insert factory (Users):", e)
            case "Orders":
                try:
                    self.__insert_into_orders(*args)
                except Exception as e:
                    print("Error in insert factory (Orders):", e)
                
    def update_user_data(self, user_id: int, name: str = None, age: int = None, email: str = None):
        user =  self.cursor.execute(f"SELECT * FROM Users WHERE id == {user_id}").fetchone()
        if not name:
            name = user[1]
        if not age:
            age = user[2]
        if not email:
            email = user[3]
            
        assert isinstance(user_id, int)
        assert isinstance(name, str)
        assert isinstance(age, int)
        assert isinstance(email, str)
        
        
        if not user:
            return f"User was not found with id {user_id}"
        
            
        assert age > 0
        
        try:
            self.cursor.execute(f"UPDATE Users SET user_name = '{name}', age = {age}, email = '{email}' WHERE id == {user_id}")
            self.database.commit()
        except Exception as e:
            print("Error has occured in update_user_data:", e)
            
    def delete_user_by_id(self, user_id: int):
        assert isinstance(user_id, int)
        
        user =  self.cursor.execute(f"SELECT * FROM Users WHERE id == {user_id}").fetchone()
        
        if not user:
            raise Exception(f'User with id {user_id} was not found')
        
        try:
            self.cursor.execute(f"DELETE FROM Users WHERE id == {user_id}")
            self.database.commit()
        except Exception as e:
            print(f"Error has occured in delete_user_by_id:", e)
            return
        
                
    def get_user_older_than(self, older_than: int):
        assert isinstance(older_than, int)
        return self.cursor.execute(f"SELECT * FROM Users WHERE age > {older_than}").fetchall()
    
    def get_user_order(self, userid: int):
        assert isinstance(userid, int)
        return self.cursor.execute(f"SELECT * FROM Orders WHERE user_id == {userid}").fetchall()
    
    def get_total_price_of_all_orders(self):
        user_amount = self.cursor.execute("SELECT max(id) FROM Users").fetchall()[0][0]
        user_with_order = pd.Series()
        
        for userid in range(user_amount):
            orders = self.cursor.execute(f"SELECT * FROM Orders WHERE user_id == {userid}").fetchall()
            for order in orders:
                try:
                    user_with_order[userid] += order[3]
                    continue
                except KeyError:
                    user_with_order[userid] = order[3]
                
        for userid, amount in user_with_order.items():
            yield f"Пользователь с id {userid} совершил заказы на сумму {amount}"
                    

In [18]:
database = DatabaseInterface('data.db')

def insert_test_data(amount: int):
    last_user = database.query("SELECT max(id) FROM Users").fetchall()[0][0]
    for i in range(amount):
        database.insert_factory("Users", faker.name(), random.randint(21, 70), faker.email())
        
        order_userid = random.randint(1, last_user)
        if database.query(f"SELECT * FROM Users WHERE id == {order_userid}").fetchone():
            database.insert_factory("Orders", order_userid, "2024-01-08 23:44:33", random.randint(100, 100000))

insert_test_data(3)

In [20]:
USER_OLDER_THAN = random.randint(0, 100)
USER_ID = random.randint(1, database.query("SELECT max(id) FROM Users").fetchall()[0][0])
print(f"Список пользователей старше {USER_OLDER_THAN} лет:\n{database.get_user_older_than(USER_OLDER_THAN)}")
print(f"Список заказов, принадлежащих пользователю с id {USER_ID}:\n{database.get_user_order(USER_ID)}")

Список пользователей старше 17 лет:
[(2, 'Тарасова Евдокия Леоновна', 38, 'zhanna_12@example.net'), (3, 'Мухин Прокл Геннадиевич', 41, 'fedotovemeljan@example.com'), (4, 'Николай Харлампович Устинов', 70, 'vlasovterenti@example.org'), (5, 'Селихов Марьян Дементьев', 34, 'Maryan@gmail.com'), (6, 'Аксенова Фёкла Артемовна', 53, 'dobromisl_15@example.com'), (7, 'Андрон Бориславович Бобылев', 50, 'anike_1991@example.net'), (8, 'Романова Анастасия Васильевна', 42, 'moise_85@example.net'), (9, 'Василиса Макаровна Федорова', 26, 'sitnikovprov@example.org'), (10, 'Святослав Захарьевич Зыков', 32, 'ermolafedoseev@example.com'), (11, 'Пелагея Наумовна Морозова', 38, 'gordeevaksenija@example.net'), (12, 'Колобов Максим Герасимович', 27, 'eduard_07@example.net'), (13, 'Зоя Ильинична Чернова', 68, 'stepanovladimir@example.org')]
Список заказов, принадлежащих пользователю с id 9:
[(10, 9, '2024-01-08 23:44:33', 84882)]


In [19]:
for i in list(database.get_total_price_of_all_orders()):
    print(i)

Пользователь с id 2 совершил заказы на сумму 41678
Пользователь с id 3 совершил заказы на сумму 75676
Пользователь с id 5 совершил заказы на сумму 77753
Пользователь с id 7 совершил заказы на сумму 87609
Пользователь с id 9 совершил заказы на сумму 84882


In [16]:
# обновление данных пользователя
# database.update_user_data(5, name="Селихов Марьян Дементьев", email="Maryan@gmail.com")

In [17]:
# delete user from table Users
# database.delete_user_by_id(1)